### Convergence of the orthogonal iteration algorithm

Note: previously run on Julia Version 1.2.0 (2019-08-20)

#### Q1. Write code to create a matrix in $\mathbb{R}^{n*n}$ of size = 8 with eigenvalues 1, 0.2, 0.05, 0.017, 0.0085, 0.0042, 0.0021, 0.0011

In [ ]:
using Base
using LinearAlgebra
import Pkg; Pkg.add("Plots")
using Plots
using Printf
using Random

In [ ]:
D = zeros(8,8)
D[1,1] = 1
D[2,2] = 0.2
D[3,3] = 0.05
D[4,4] = 0.017
D[5,5] = 0.0085
D[6,6] = 0.0042
D[7,7] = 0.0021
D[8,8] = 0.0011

rng = MersenneTwister(2019)

P = rand(rng,8,8)
A = P*D*inv(P) # by diagnonalization theorem
println("Matrix A is")
A

In [ ]:
println("Eigenvalues of matrix A are")
eigvals(A)

#### Q2. Implement the orthogonal iteration algorithm. Print the values along the diagnoal of $R_k$ at each iteartion $k$ for $k = 1, ..., 5$. Print each number using at most 4 significant digits.

#### Q4. Assume that $\left\|A_k(p:n,1:p-1)\right\|_2 $ and $\left\|A_k(p+1:n,1:p)\right\|_2$ are very small. Show that the entry $A_k(p,p)$ is very close to an eigenvalues of A. 

#### Q5. Considering entry $p$ along the diagonal, plot the convergence of the pth eigenvalue. Choose $p = 1, 2, and 3$, Use a semi-logarithmic plot. We would expect the following theoretical rate of convergence at step $k$ for entry $p$:  

<img src="IMG_2478.jpg" width="500">

In [ ]:
# Q2 Setup

Qk = rand(rng,8,8)
Ak = zeros(8,8)
block = zeros(4,4)
y_2norm = []
y_p1 = []
y_p2 = []
y_p3 = []

In [ ]:
for k = 1:5
    println("===Itearation: ", k, "===")
    global Qk
    Rk = A*Qk
    Q,R = qr(Qk)
    println("## Q2 ## ")
    println("R[1,1] is ", round(R[1,1], sigdigits=4))
    println("R[2,2] is ", round(R[2,2], sigdigits=4))
    println("R[3,3] is ", round(R[3,3], sigdigits=4))
    println("R[4,4] is ", round(R[4,4], sigdigits=4))
    println("R[5,5] is ", round(R[5,5], sigdigits=4))
    println("R[6,6] is ", round(R[6,6], sigdigits=4))
    println("R[7,7] is ", round(R[7,7], sigdigits=4))
    println("R[8,8] is ", round(R[8,8], sigdigits=4))
    Qk = Q
    
    Ak = transpose(Qk)*A*Qk
    
    # Q3 Consider block [5:8, 1:4]
    block = Ak[5:8, 1:4]
    append!(y_2norm, opnorm(block,2))
    
    # Q4 Observe diagonal of Ak
    println("## Q4 ## ")
    
    # Note: the preblock here means A_k[p:n, 1:p-1]
    # postblock here means A_k[p+1:n, 1:p]
    # On boundary, when p=1, we only have block A_k[2:8, 1]
    # when p=8, we only have block A_k[8,1:7]
    for p = 1:8
        if p == 1
            postblock = Ak[2:8, 1]
            m = @sprintf " When p=1, block 2-norm is %1.4f, Ak diag val is %1.4f"\
            norm(postblock,2) Ak[1,1]
            println(m)
        elseif p == 8
            postblock = Ak[8, 1:7]
            m = @sprintf " When p=8, block 2-norm is %1.4f, Ak diag val is %1.4f"\
            norm(postblock,2) Ak[8,8]
        else
            preblock = Ak[p:8, 1:(p-1)]
            postblock = Ak[(p+1):8, 1:p]
            m = @sprintf \
            " When p = %1.0f, preblock norm is %1.4f, postblock norm is %1.4f, Ak diag is %1.4f"\
            p norm(preblock,2) norm(postblock,2) Ak[p,p]
            println(m)
        end
    end
    
    # Q5 Convergence of pth eval: p=1,2,3
    for p = 1:3
        if p == 1
            append!(y_p1, abs(1-Ak[1,1]))
        elseif p == 2
            append!(y_p2, abs(0.2-Ak[2,2]))
        else
            append!(y_p3, abs(0.05-Ak[3,3]))
        end
    end
end


#### Q3. Consider the block $(p+1:n, 1:p)$ in the matrix $A_k = Q_k^T A Q_k$, plot the 2-norm of this block as a function of $k$ for $p = 4$. Compare with the analytical estiate, which states that it should decay like $\mid \frac{\lambda_{p+1}}{\lambda_p} \mid ^ k$

In [ ]:
plot(y_2norm; yscale =: log)

In [ ]:
# Q5 plotting
plot(y_p1; yscale=: log)
plot(y_p2; yscale=: log)
plot(y_p3; yscale=: log)

Observation: At early print of Q4 result, we would see quite large block 2-norms, and the diag value is not quite accurate; at late stages, the norms become much smaller with Ak diag being closer to eigenvalues. 

Other written analyses:

![](IMG_2480.jpg)
![](IMG_2481.jpg)
![](IMG_2482.jpg)